In [1]:
from ophyd_async.core import DeviceCollector 
from ophyd_async.epics.motion.motor import Motor
from ophyd_async.core import Device
from ophyd_async.epics.signal import epics_signal_rw
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky import RunEngine
from bluesky.plans import count, scan 
from ophyd.sim import det1
from bluesky import utils
from bluesky.utils import Msg
from dodal.beamlines.beamline_utils import set_directory_provider
from ophyd_async.core._providers import StaticDirectoryProvider
from ophyd_async.epics.signal import (
    epics_signal_r,
    epics_signal_rw,
    epics_signal_x,
    epics_signal_w
)

import asyncio
from epics import caget, caput,cainfo
from enum import Enum
loop = asyncio.get_event_loop()
bec = BestEffortCallback()
RE = RunEngine({})

In [ ]:
import os 
os.chdir("/scratch/wvx67826/p99Bluesky/src/")

In [ ]:
from p99Bluesky.devices.p99.sample_stage import SampleAngleStage
from p99Bluesky.devices.stages import ThreeAxisStage
from p99Bluesky.devices.epics.setReadOnlyMotor import SetReadOnlyMotor


In [ ]:
with DeviceCollector():
    sample_angle_stage = SampleAngleStage('BL99P-MO-STAGE-01:',name = "sample_angle_stage")
#result = asyncio.create_task(x.connect())

In [ ]:
result1 = asyncio.create_task(sample_angle_stage.theta.user_setpoint.get_value())
result2 = asyncio.create_task(sample_angle_stage.roll.read())
result3 = asyncio.create_task(sample_angle_stage.pitch.read())

In [ ]:
print(result1,"\n",result2,"\n",result3,"\n", )


In [ ]:
asyncio.all_tasks(sample_angle_stage.roll.set(0.12))
asyncio.all_tasks(sample_angle_stage.theta.set(0.21))
asyncio.all_tasks(sample_angle_stage.pitch.set(0.11))

In [ ]:
print (result1,"\n\n",result2 ,"\n\n",result3)

In [ ]:
RE(count([sample_angle_stage.theta,sample_angle_stage.roll, sample_angle_stage.pitch],5),bec)

In [ ]:
result1 = asyncio.create_task(sample_angle_stage.roll.read_configuration())
result2 = asyncio.create_task(sample_angle_stage.roll.read_configuration())
result3 = asyncio.create_task(sample_angle_stage.roll.read_configuration())

In [ ]:
print (result1,"\n\n",result2 ,"\n\n",result3)

In [ ]:
xyz_pv = 'BL99P-MO-STAGE-02:'

In [ ]:
with DeviceCollector():
    sample_xyz_stage =ThreeAxisStage(xyz_pv,"sample_xyz_stage")

In [ ]:
sample_xyz_stage.x.setpoint = epics_signal_rw(float, xyz_pv + "X")
sample_xyz_stage.y.setpoint = epics_signal_rw(float, xyz_pv + "Y")
sample_xyz_stage.z.setpoint = epics_signal_rw(float, xyz_pv + "Z")
asyncio.create_task(sample_xyz_stage.x.connect())
asyncio.create_task(sample_xyz_stage.y.connect())
asyncio.create_task(sample_xyz_stage.z.connect())

In [ ]:
result1 = asyncio.create_task(sample_xyz_stage.x.read())
result2 = asyncio.create_task(sample_xyz_stage.y.read())
result3 = asyncio.create_task(sample_xyz_stage.z.read())

In [ ]:
print(result1,"\n",result2,"\n",result3,"\n", )

In [ ]:
result1 = asyncio.create_task(sample_xyz_stage.x.read_configuration())
result2 = asyncio.create_task(sample_xyz_stage.y.read_configuration())
result3 = asyncio.create_task(sample_xyz_stage.z.read_configuration())

In [ ]:
print(result1,"\n",result2,"\n",result3,"\n", )

In [ ]:
RE(count([sample_xyz_stage.x, sample_xyz_stage.y, sample_xyz_stage.z ],3),bec)

In [ ]:
caget('BL99P-EA-DET-03:CAM:AndorADCSpeed_RBV')

In [ ]:
setpoint = epics_signal_rw(Enum, 'BL99P-MO-STAGE-02:MP:SELECT')

In [ ]:
caput('BL99P-MO-STAGE-02:MP:SELECT', Color.e)

In [2]:
class p99StageSelections(str, Enum):
    Empty = ("Empty",)
    Mn5um = ("Mn 5um",)
    Fe = ("Fe (empty)",)
    Co5um = ("Co 5um",)
    Ni5um = ("Ni 5um",)
    Cu5um = ("Cu 5um",)
    Zn5um = ("Zn 5um",)
    Zr = ("Zr (empty)",)
    Mo = ("Mo (empty)",)
    Rh = ("Rh (empty)",)
    Pd = ("Pd (empty)",)
    Ag = ("Ag (empty)",)
    Cd25um = ("Cd 25um",)
    W = ("W (empty)",)
    Pt = ("Pt (empty)",)
    User = ("User",)

class FilterMotor(Device):
    def __init__(self, prefix: str, name: str):
        self.user_setpoint = epics_signal_rw(p99StageSelections, 'BL99P-MO-STAGE-02:MP:SELECT')
        super().__init__(name=name)

In [3]:
with DeviceCollector():
    filterWheel =  FilterMotor('BL99P-MO-STAGE-02:MP:SELECT', "filterWheel")

In [ ]:
setpoint = epics_signal_rw(p99StageSelections, 'BL99P-MO-STAGE-02:MP:SELECT')
result1 = asyncio.create_task(setpoint.connect())
#

In [8]:
result1= asyncio.gather(filterWheel.user_setpoint.set(p99StageSelections.Co5um))
result2 = asyncio.create_task(filterWheel.user_setpoint.read())

In [9]:
print(result1,"\n",result2,"\n" )

<_GatheringFuture finished result=[None]> 
 <Task finished name='Task-15' coro=<SignalR.read() done, defined at /scratch/wvx67826/myenv/lib/python3.11/site-packages/ophyd_async/core/signal.py:28> result={'filterWheel-user_setpoint': {'alarm_severity': 0, 'timestamp': 1713863975.657885, 'value': <GeneratedCho...5um: 'Co 5um'>}}> 

